In [11]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import ta
from ta import add_all_ta_features
from ta.utils import dropna
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from main import add_classic_indicators
from alpha101 import add_artificial_variables
from arch import arch_model
import itertools
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

nasdaq_tickers = ['TMUSR', 'AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'GOOG', 'INTC', 'NVDA', 'ADBE',
               'PYPL', 'CSCO', 'NFLX', 'PEP', 'TSLA', 'CMCSA', 'AMGN', 'COST', 'TMUS', 'AVGO',
               'TXN', 'CHTR', 'QCOM', 'GILD', 'SBUX', 'INTU', 'VRTX', 'MDLZ', 'ISRG', 'FISV',
               'BKNG', 'ADP', '', 'ATVI', 'AMD', 'JD', 'MU', 'AMAT', 'ILMN', 'ADSK',
               'CSX', 'MELI', 'LRCX', 'ADI', 'ZM', 'BIIB', 'EA', 'KHC', 'WBA', 'LULU',
               'EBAY', 'MNST', 'DXCM', 'EXC', 'BIDU', 'XEL', 'WDAY', 'DOCU', 'SPLK', 'ORLY',
               'NXPI', 'CTSH', 'KLAC', 'SNPS', 'SGEN', 'ASML', 'IDXX', 'CSGP', 'CTAS', 'VRSK',
               'MAR', 'CDNS', 'PAYX', 'ALXN', 'MCHP', 'SIRI', 'ANSS', 'VRSN', 'FAST', 'BMRN',
               'XLNX', 'INCY','DLTR', 'SWKS', 'ALGN', 'CERN', 'CPRT', 'CTXS', 'TTWO', 'MXIM',
               'CDW', 'CHKP', 'WDC', 'ULTA', 'NTAP', 'FOXA', 'LBTYK']

ModuleNotFoundError: No module named 'talib'

In [6]:
data = yf.Ticker(nasdaq_tickers[1]).history(period='5y').drop(columns=['Dividends','Stock Splits'])
data = add_classic_indicators(data)
data = add_artificial_variables(data)
data.tail()

/Users/louisdelacour/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/louisdelacour/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/Users/louisdelacour/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/louisdelacour/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/Users/louisdelacour/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/louisdelacour/opt/anaconda3/lib/python3.9/site-packages/ta/trend.py:785: 

,Open,High,Low,Close,Volume,adx 4,ema7,ema30,ema50,ema100,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-17 00:00:00-05:00,146.429993,151.479996,146.149994,150.720001,80389400.0,44.394049,147.835122,146.465855,147.758544,150.187621,...,0.291142,-0.390621,0.681763,-3.350636,-0.118653,-3.231983,147.128783,1.297136,1.288795,262.353563
2022-11-18 00:00:00-05:00,152.309998,152.699997,149.970001,151.289993,74794600.0,47.244716,148.698840,146.777090,147.897032,150.209450,...,0.517986,-0.208900,0.726886,-4.279404,-0.950803,-3.328601,147.630106,0.378179,0.377466,263.723909
2022-11-21 00:00:00-05:00,150.160004,150.369995,147.720001,148.009995,58724100.0,39.098309,148.526629,146.856632,147.901462,150.165896,...,0.511625,-0.064795,0.576419,-6.523844,-2.065412,-4.458433,147.652995,-2.168021,-2.191868,255.838299
2022-11-22 00:00:00-05:00,148.130005,150.419998,146.929993,150.179993,51804100.0,29.834950,148.939970,147.071042,147.990816,150.166176,...,0.617974,0.071759,0.546215,-9.025155,-3.457360,-5.567795,147.832459,1.466116,1.455472,261.055301
2022-11-23 00:00:00-05:00,149.449997,151.830002,149.339996,151.070007,58237000.0,27.968483,149.472479,147.329040,148.111569,150.184073,...,0.741722,0.205751,0.535971,-10.354987,-4.836886,-5.518101,148.381117,0.592632,0.590883,263.195030


In [8]:
pip install talib

ERROR: Could not find a version that satisfies the requirement talib (from versions: none)
ERROR: No matching distribution found for talib
Note: you may need to restart the kernel to use updated packages.
